# Dev

In [1]:
# Run this if you are modifying external functions
import sys
sys.dont_write_bytecode=True

%load_ext autoreload

# Add this before any functions you are importing that you have changed
%autoreload

# Setup

In [4]:
## Data folder
import os
def file_path_prefix(file_prefix):
    folder = f'../data/{file_prefix}'
    os.makedirs(folder, exist_ok=True)
    return f'{folder}/{file_prefix}'

# Inputs

In [9]:
# Inputs
## Name of the area of interest
aoi_name='amsterdam-test'

## Path to polygon file the area you want data for
## From https://geojson.io/#new&map=15.37/52.374412/4.905746
aoi_url = '../data/amsterdam-test.geojson'


# Get Polygon for AOI

In [ ]:
# load boundary
import geopandas as gpd

# If you are using an SSO accout, you need to be authenticated first
# !aws sso login
aoi_gdf = gpd.read_file(aoi_url, driver='GeoJSON')

aoi_gdf = aoi_gdf.to_crs(epsg=4326)

## Write to file

file_path = f'{file_path_prefix(aoi_name)}-boundary.geojson'
aoi_gdf.to_file(file_path, driver='GeoJSON')
print(f'File saved to {file_path}')



## Get area in km2 of the city rounded to the nearest integer
aoi_gdf_area = aoi_gdf['geometry'].to_crs(epsg=3857).area/ 10**6 # in km2
aoi_gdf_area = round(aoi_gdf_area.values[0], 3)
print(f'Area: {aoi_gdf_area} sqkm')

# LULC

In [ ]:
# Define function
from dask.diagnostics import ProgressBar
import xarray as xr
import xee
import ee

from city_metrix.layers import layer, Layer

class OpenUrban(Layer):
    def __init__(self, band='b1', **kwargs):
        super().__init__(**kwargs)
        self.band = band

    def get_data(self, bbox):
        dataset = ee.ImageCollection("projects/wri-datalab/cities/OpenUrban/OpenUrban_LULC")
        ## It is important if the cif code is pulling data from GEE to take the maximum value where the image tiles overlap


        # Check for data
        if dataset.filterBounds(ee.Geometry.BBox(*bbox)).size().getInfo() == 0:
            print("No Data Available")
        else:
            ulu = ee.ImageCollection(dataset
                                     .filterBounds(ee.Geometry.BBox(*bbox))
                                     .select(self.band)
                                     .reduce(ee.Reducer.firstNonNull())
                                     .rename('lulc')
                                     )

        data = layer.get_image_collection(ulu, bbox, 1, "urban land use").lulc

        return data
    


In [ ]:
# Load data
aoi_LULC = OpenUrban().get_data(aoi_gdf.total_bounds)

# Get resolution of the data
aoi_LULC.rio.resolution()

In [ ]:
# Define reclassification
from enum import Enum

# From https://gfw.atlassian.net/wiki/spaces/CIT/pages/872349733/Surface+characteristics+by+LULC#Major-update-to-LULC-codes
class OpenUrbanClass(Enum):
    GREEN_SPACE_OTHER = 110.0
    BUILT_UP_OTHER = 120.0
    BARREN = 130.0
    PUBLIC_OPEN_SPACE = 200.0
    WATER = 300.0
    PARKING = 400.0
    ROADS = 500.0
    BUILDINGS_UNCLASSIFIED = 600.0
    BUILDINGS_UNCLASSIFIED_LOW_SLOPE = 601.0
    BUILDINGS_UNCLASSIFIED_HIGH_SLOPE = 602.0
    BUILDINGS_RESIDENTIAL = 610.0
    BUILDINGS_RESIDENTIAL_LOW_SLOPE = 611.0
    BUILDINGS_RESIDENTIAL_HIGH_SLOPE = 612.0
    BUILDINGS_NON_RESIDENTIAL = 620.0
    BUILDINGS_NON_RESIDENTIAL_LOW_SLOPE = 621.0
    BUILDINGS_NON_RESIDENTIAL_HIGH_SLOPE = 622.0

# Note, it seems these have to be in the same order as the OpenUrbanClass
reclass_map = {
    OpenUrbanClass.GREEN_SPACE_OTHER.value: 5.0,
    OpenUrbanClass.BUILT_UP_OTHER.value: 1.0,
    OpenUrbanClass.BARREN.value: 6.0,
    OpenUrbanClass.PUBLIC_OPEN_SPACE.value: 5.0,
    OpenUrbanClass.WATER.value: 7.0,
    OpenUrbanClass.PARKING.value: 1.0,
    OpenUrbanClass.ROADS.value: 1.0,
    OpenUrbanClass.BUILDINGS_UNCLASSIFIED.value: 2.0,
    OpenUrbanClass.BUILDINGS_UNCLASSIFIED_LOW_SLOPE.value: 2.0,
    OpenUrbanClass.BUILDINGS_UNCLASSIFIED_HIGH_SLOPE.value: 2.0,
    OpenUrbanClass.BUILDINGS_RESIDENTIAL.value: 2.0,
    OpenUrbanClass.BUILDINGS_RESIDENTIAL_LOW_SLOPE.value: 2.0,
    OpenUrbanClass.BUILDINGS_RESIDENTIAL_HIGH_SLOPE.value: 2.0,
    OpenUrbanClass.BUILDINGS_NON_RESIDENTIAL.value: 2.0,
    OpenUrbanClass.BUILDINGS_NON_RESIDENTIAL_LOW_SLOPE.value: 2.0,
    OpenUrbanClass.BUILDINGS_NON_RESIDENTIAL_HIGH_SLOPE.value: 2.0,
    }

reclass_map

In [ ]:
# Reclassify
from xrspatial.classify import reclassify

aoi_LULC_to_solweig = reclassify(aoi_LULC, bins=list(reclass_map.keys()), new_values=list(reclass_map.values()), name='lulc')


In [ ]:
# Check results
aoi_LULC_counts = aoi_LULC.groupby(aoi_LULC).count().to_dataframe()
print(aoi_LULC_counts)

aoi_LULC_V2_to_solweig_counts = aoi_LULC_V2_to_solweig.groupby(aoi_LULC_V2_to_solweig).count().to_dataframe()
print(aoi_LULC_V2_to_solweig_counts)

In [ ]:
# Remove zeros
remove_value = 0

def count_occurrences(data, value):
    return data.where(data==value).count().item()

count = count_occurrences(aoi_LULC_to_solweig, remove_value)

if count > 0:
    print(f'Found {count} occurrences of the value {remove_value}. Removing...')
    aoi_LULC_to_solweig = aoi_LULC_to_solweig.where(aoi_LULC_to_solweig!=remove_value, drop=True)
    count = count_occurrences(aoi_LULC_to_solweig, remove_value)
    print(f'There are {count} occurrences of the value {remove_value} after removing.')
else:
    print(f'There were no occurrences of the value {remove_value} found in data.')


In [ ]:
# Save data to file
file_path = f'{file_path_prefix(aoi_name)}-lulc.tif'
aoi_LULC_V2_to_solweig.astype('float32').rio.to_raster(raster_path=file_path, driver="COG")
print(f'File saved to {file_path}')

# High Resolution 1m Global Canopy Height Maps

In [ ]:
from city_metrix.layers import TreeCanopyHeight

# Load layer
aoi_TreeCanopyHeight = TreeCanopyHeight().get_data(aoi_gdf.total_bounds)

# Get resolution of the data
aoi_TreeCanopyHeight.rio.resolution()

In [ ]:
aoi_TreeCanopyHeight.rio.crs

In [ ]:
# Save data to file
file_path = f'{file_path_prefix(aoi_name)}-TreeCanopyHeight.tif'
aoi_TreeCanopyHeight.astype('float32').rio.to_raster(raster_path=file_path, driver="COG")
print(f'File saved to {file_path}')

In [ ]:
aoi_TreeCanopyHeight.plot()

# Building footprints

In [ ]:
from city_metrix.layers import OvertureBuildings

# Load layer
aoi_OvertureBuildings = OvertureBuildings().get_data(aoi_gdf.total_bounds)

# Get row and column count
aoi_OvertureBuildings.shape


In [ ]:
# Save data to file
file_path = f'{file_path_prefix(aoi_name)}-OvertureBuildings.geojson'
aoi_OvertureBuildings.to_file(file_path, driver='GeoJSON')
print(f'File saved to {file_path}')


# DSM

In [ ]:
from city_metrix.layers import AlosDSM

aoi_AlosDSM = AlosDSM().get_data(aoi_gdf.total_bounds)

# Get resolution of the data
aoi_AlosDSM.rio.resolution()

In [ ]:
# Save data to file
file_path = f'{file_path_prefix(aoi_name)}-aoi_AlosDSM.tif'
aoi_AlosDSM.rio.to_raster(raster_path=file_path, driver="COG")
print(f'File saved to {file_path}')

In [ ]:
from rasterio.enums import Resampling

dsm_1m = aoi_AlosDSM.rio.reproject(
            dst_crs=aoi_AlosDSM.rio.crs,
            resolution=1,
            resampling=Resampling.bilinear
        )

# Get resolution of the data
dsm_1m.rio.resolution()

In [ ]:
# Save data to file
file_path = f'{file_path_prefix(aoi_name)}-aoi_AlosDSM_1m.tif'
dsm_1m.rio.to_raster(raster_path=file_path, driver="COG")
print(f'File saved to {file_path}')

# DEM

In [ ]:
from city_metrix.layers import NasaDEM

aoi_NasaDEM = NasaDEM().get_data(aoi_gdf.total_bounds)

# Get resolution of the data
aoi_NasaDEM.rio.resolution()

In [ ]:
# Save data to file
file_path = f'{file_path_prefix(aoi_name)}-aoi_NasaDEM.tif'
aoi_NasaDEM.rio.to_raster(raster_path=file_path, driver="COG")
print(f'File saved to {file_path}')

In [ ]:
dem_1m = aoi_NasaDEM.rio.reproject(
            dst_crs=aoi_NasaDEM.rio.crs,
            resolution=1,
            resampling=Resampling.bilinear
        )

# Get resolution of the data
dem_1m.rio.resolution()

In [ ]:
# Save data to file
file_path = f'{file_path_prefix(aoi_name)}-aoi_NasaDEM_1m.tif'
dem_1m.rio.to_raster(raster_path=file_path, driver="COG")
print(f'File saved to {file_path}')

# Building height

In [ ]:
aoi_height = aoi_AlosDSM - aoi_NasaDEM

# Get resolution of the data
aoi_height.rio.resolution()

In [ ]:
from exactextract import exact_extract

aoi_OvertureBuildings = aoi_OvertureBuildings.to_crs(aoi_AlosDSM.rio.crs)

aoi_OvertureBuildings['AlosDSM_max'] = exact_extract(aoi_AlosDSM, aoi_OvertureBuildings, ["max"], output='pandas')['max']
aoi_OvertureBuildings['NasaDEM_max'] = exact_extract(aoi_NasaDEM, aoi_OvertureBuildings, ["max"], output='pandas')['max']
aoi_OvertureBuildings['height_max'] = exact_extract(aoi_height, aoi_OvertureBuildings, ["max"], output='pandas')['max']

# Get row and column count
aoi_OvertureBuildings.shape


In [ ]:
# Write to file
file_path = f'{file_path_prefix(aoi_name)}-BuildingHights.geojson'
aoi_OvertureBuildings.to_file(file_path, driver='GeoJSON')
print(f'File saved to {file_path}')

In [ ]:

def rasterize_polygon(self, gdf, snap_to):
        if gdf.empty:
            raster = np.full(snap_to.shape, 0, dtype=np.int8)
            raster = xr.DataArray(raster, dims=snap_to.dims, coords=snap_to.coords)

            return raster.rio.write_crs(snap_to.rio.crs, inplace=True)

        raster = make_geocube(
            vector_data=gdf,
            measurements=["Value"],
            like=snap_to,
            fill=np.int8(0)
        ).Value

        return raster.rio.reproject_match(snap_to)










# ERA5

In [ ]:
#TODO

# LULCv2

In [ ]:
from city_metrix.layers import SmartSurfaceLULC

aoi_smart_surface_lulc = SmartSurfaceLULC().get_data(aoi_gdf.total_bounds)


In [ ]:
from matplotlib.colors import ListedColormap
from enum import Enum

# Define the lulcPaletteV3
lulcPaletteV3 = [
    'b2df8a',
    'cccccc',
    '808080',
    '33a02c',
    'a6cee3',
    '000000',
    '493910',
    '5e4915',
    '735a19',
    '886a1e',
    '9d7a23',
    'b28b27',
    'f5efba',
    'c9beb6'
]

# Define the SsLulcClass Enum
class SsLulcClass(Enum):
    GREEN_SPACE_OTHER = 1
    BUILT_UP_OTHER = 2
    BARREN = 3
    OPEN_SPACE = 10
    WATER = 20
    ROADS = 30
    NON_CLASSIFIED_BUILDINGS = 40
    RESIDENTIAL_HIGH_SLOPE = 41
    NON_RESIDENTIAL_HIGH_SLOPE = 42
    UNCLASSIFIED_HIGH_SLOPE = 43
    RESIDENTIAL_LOW_SLOPE = 44
    NON_RESIDENTIAL_LOW_SLOPE = 45
    UNCLASSIFIED_LOW_SLOPE = 46
    PARKING = 50

# Create a colormap dictionary
colormap = {cls.value: f'#{color}' for cls, color in zip(SsLulcClass, lulcPaletteV3)}

# Create a ListedColormap for matplotlib
cmap = ListedColormap([colormap[key] for key in colormap.keys()])

In [ ]:
cmap

In [ ]:
# Convert hex color codes to RGBA tuples
def hex_to_rgba(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) + (255,)

rgba_colormap = {key: hex_to_rgba(value) for key, value in colormap.items()}

rgba_colormap

In [ ]:
rgba_colormap

In [ ]:
# Save the DataArray with the colormap as a COG
file_path = f'{file_path_prefix(aoi_name)}-smart_surface_lulc_styled.tif'

aoi_smart_surface_lulc.rio.to_raster(
    file_path,
    driver="COG",
    dtype="uint8",
    compress="deflate")

print(f'File saved to {file_path}')

In [ ]:
import rasterio

with rasterio.Env():

    with rasterio.open(file_path) as src:
        shade = src.read(1)
        meta = src.meta

    with rasterio.open(file_path, 'w', **meta) as dst:
        dst.write(shade, indexes=1)
        dst.write_colormap(
            1, rgba_colormap)
        cmap = dst.colormap(1)